In [99]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4

cpu


In [100]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [101]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

tensor([80,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


In [102]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n: ]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

x, y = get_batch('train')
print('inputs: ')
print(x)
print('targets: ')
print(y)

tensor([ 78372,  28729, 101713,  56876])
inputs: 
tensor([[74, 58,  0, 68, 59,  1, 34, 62],
        [72, 72, 62, 68, 67,  1, 73, 61],
        [ 1, 72, 74, 72, 69, 58, 67, 57],
        [64, 67, 68, 76,  1, 73, 61, 54]])
targets: 
tensor([[58,  0, 68, 59,  1, 34, 62, 66],
        [72, 62, 68, 67,  1, 73, 61, 54],
        [72, 74, 72, 69, 58, 67, 57, 58],
        [67, 68, 76,  1, 73, 61, 54, 73]])


In [103]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(28)
when input is tensor([80,  1, 28]) target is tensor(39)
when input is tensor([80,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1, 28, 39, 42, 39, 44]) target is tensor(32)
when input is tensor([80,  1, 28, 39, 42, 39, 44, 32]) target is tensor(49)


In [104]:
import torch
import torch.nn as nn  # Corrected import statement

class BigramLanguageModel(nn.Module):  # Corrected 'Module' capitalization
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)  # Fixed '==' to '='

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None: 
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index if (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars  = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)
    
        


!pMP5XQtD,Y3GZOH?[ !Gu224Y*Lk:4Q8zLp﻿k?tiFzU'hBw8Cq9Q8W:OhM*eo:WW_o2Bx3ePCWAIi?3O)GMy!ZShj]dDIZ(Zn,w?3&lsLkX4dbNYNL?NEl?Q3
QqDRR*39SPI;
)cNe1e1 w4SJF0ni,Bi,'UkNt[T!WAIn4 SBm?Q8q3.8PD2?o(b7Uac'3JEA&0e&; R.M)m(n:_Mun.4oAOR:dnaig8znd,BxR1xJ7R1.X"
yIMS9Q?WoW5T
Pwx]NEv9i9XxLY8fjzetWxJ9Q7btj:JSr]5":tJhF-dBe
vd)sQIF 1K?WZsbDjBqa?NebG﻿,pa;N]Nv 7bZoY0Cj7Rc﻿(-HWLdK"x]NeNEAxcVHfIIZ*tr!Ln4L04(BEyrJoTWno,'jOgkHKkHKL&lR
d4Q88hX)haG!4W?'&:;!ucQnMR2bl6rf!TsP1vHD6rXxSa2Acx] 5-H9TP?'U.Z9QqgnrT 'D9QvEnk
Wzj[LTfr*[


In [108]:
# create a Pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# After the training loop is complete, print the final loss
print("Final Loss:", loss.item())


tensor([ 64255, 102492, 158509,  15454])
tensor([ 90469, 155353, 137606, 172524])
tensor([99205, 51488, 52215, 97516])
tensor([ 69073,  30797, 158719, 123541])
tensor([ 30966,  78721,  73298, 155146])
tensor([ 55357, 132544, 149574,  22467])
tensor([  5210, 125930,  49788, 120700])
tensor([ 64196, 134403, 101158,  11674])
tensor([125765,  18415, 178463, 151029])
tensor([   154,   5406,  32229, 104086])
tensor([ 28238,  42361,  59065, 119064])
tensor([ 81254,  31504, 121769,  47624])
tensor([ 47626, 167528, 158212,  28448])
tensor([ 94957,  62961, 108670,  13991])
tensor([139641,  37881,   3273,  93874])
tensor([ 21848,  94787,  28525, 158597])
tensor([   149, 145994, 160591,  16834])
tensor([ 39860, 175101,  28676,  48933])
tensor([179191, 151690,   3399, 128797])
tensor([ 18074, 110618, 165078,  73903])
tensor([38413, 23728, 77205, 46002])
tensor([153778,  78108,  94696,  34957])
tensor([80024, 44947, 23836, 32422])
tensor([110604, 131034,  78408, 154528])
tensor([  5112, 103781, 1112